In [121]:
import cospaper as cosp
import sys
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.nonparametric.kernel_regression as krg
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import  summary_table
from helper_functions import wl_to_v, v_to_wl, deltawl_to_v, v_to_deltawl
#from LARS_branding import lbrand
#import pyqt_fit as pqf
#import pyqt_fit.kernel_smoothing as smooth

In [12]:
#vdict = cosp.load_dfdict('../Products/vflis.pickle')
#wdict = cosp.load_dfdict('../Products/dfdict.pickle')
vlisdict = pd.read_pickle('../Products/vflis.pickle')
vhisdict = pd.read_pickle('../Products/vfhis.pickle')
wdict = pd.read_pickle('../Products/dfdict.pickle')

In [20]:
wdict[1].z_value = 0.020434136
vlisdict[1].z_value = 0.020434136
vhisdict[1].z_value = 0.020434136

In [29]:
frame = wdict[1]
plt.plot(frame['lambda'], frame.flam)
plt.show()

In [119]:
cosp.show_spec(inframe=frame)
plt.axvline(LyaWL * (1 + 0.020434136), color='k', linestyle='--')
plt.axis((1230, 1250, -1e-14, 1.4e-12))

9.253416e-14


(1230, 1250, -1e-14, 1.4e-12)

In [21]:
LyaWL = 1215.668

In [30]:
LyaWL * (1 + 0.020434136)

1240.5091252428479

In [132]:
wlmin_lya = 1240.18
wlpeak_lya = 1241.14
vlis = -129.848497
print 'Wave diff of Lya minumum:                             ', 1240.5091252428479 - wlmin_lya
print 'Wave diff of Lya peak:                                ', wlpeak_lya - 1240.5091252428479
print 'Velocity of Lyalpha minimum:                          ', wl_to_v(wlmin_lya, 1240.5091252428479)
print 'Lya peak velocity:                                    ', wl_to_v(wlpeak_lya, 1240.5091252428479)
print '-v_LIS:                                               ', 129.848497
print 'Sandberg 2013 redshift uncertainty in Ångström @ Lya: ', v_to_deltawl(7., 1240.5091252428479)
print 'vlis uncertainty in Ångström @Lya                     ', v_to_deltawl(22.275430, 1240.5091252428479)

Wave diff of Lya minumum:                              0.329125242848
Wave diff of Lya peak:                                 0.630874757152
Velocity of Lyalpha minimum:                           -79.5393306953
Lya peak velocity:                                     152.462799578
-v_LIS:                                                129.848497
Sandberg 2013 redshift uncertainty in Ångström @ Lya:  0.0289652512763
vlis uncertainty in Ångström @Lya                      0.0921733467481


$-v_{LIS}$ is consistent with  $v^{peak}_{Ly\alpha}$, given the uncertainties in the former and in $z$. 

In [102]:
def prettify(ax):
    ax.tick_params(length=2, labelsize=9)
    [j.set_linewidth(.6) for j in ax.spines.itervalues()]
    ax.axhline(0, color='k', ls='--', lw=.5)
    ax.axvline(0, color='k', ls='--', lw=.5)
    return ax

def LyA_vel(indata, larsno, degree=1, ax=plt.gca(), plotit=True, iterations=1):
    '''indata must be dfdict, larsno must be one of the keys in dfdict.keys()'''
    data = indata[larsno]
    LyaWL = 1215.668
    LyaWL *= (1+data.z_value)
    vels = (data['lambda'] - LyaWL)/(LyaWL) * sp.constants.c / 1000
    velidx = vels.between(-3000, 3000)
    flux = data.flam[velidx]
    vels = vels[velidx]
    errs = data.stddev[velidx]
    # For the sake of later Monte Carlo:
    #plt.hist(errs)
    errwidth = errs.std()
    errmean = errs.mean()
    print ' '
    sys.stdout.write('\rCross validating...')
    mle = krg.KernelReg(flux, vels, var_type='c', bw='cv_ls')#[20])#bw='aic')
    sys.stdout.flush()
    #print '\n'
    #print 'Estimator: '
    #print mle  #, type(mle)
    means, mfx = mle.fit()
    #realvel = vels(means.argmax())
    kernelwidth = mle.bw[0]
    kernel_widths = sp.linspace(kernelwidth / 2., kernelwidth * 1.5, 100)
    velsteps = vels.ptp() / (vels.shape[0])-1.
    vels_dict = {}
    
    count = 0
    sys.stdout.write('\r Estimating uncertainty...')
    sys.stdout.flush()
    print ' ' 
    while count < iterations:
        kaunt = count+1
        sys.stdout.flush()
        sys.stdout.write('\r Iteration: {}'.format(kaunt))
        mcdata = flux.copy()
        signs = sp.around(sp.random.random(len(mcdata)))
        signs[signs == 0] = -1
        # permute = sp.random.normal(errmean, errwidth, len(mcdata))
        permute = [sp.random.normal(loc=0, scale=errs.iloc[i]) for i in range(len(errs))]
        #print permute.shape, signs.shape, mcdata.shape
        mcdata += (permute * signs)
        mles = krg.KernelReg(mcdata, vels, var_type='c', bw=[kernelwidth])
        model, mfx = mles.fit()
        #print  vels.iloc[model.argmax()]#, model.shape,
        #continue
        # sys.stdout.write(' '.join([str(model.argmax()), str(vels)]))
        vels_dict[kaunt] = [22., vels.iloc[model.argmax()]]
        count += 1
        
    print '\n'
    vdf = pd.DataFrame.from_dict(vels_dict).T
    vdf.columns = ['Kernel', 'Centroid']
    offset = 0  # flux.median() * .08
    if plotit:
        ax.label_outer()
        ax.set_xlim(-1500, 1500)
        #ax.set_ylim(means.max()*-.2e14, means.max()*1.6e14)
        scalefactor = np.median(flux)
        ax.plot(vels, flux/scalefactor, drawstyle='steps-mid', color='k', lw=.5)
        ax.plot(vels, (means-offset)/scalefactor, lw=.8, color='orange',
                 label='LLE - CV, '+data.columns.name)
        ax.fill_between(vels, (errs)/scalefactor, color='.7', step='mid')
        ax.legend(shadow=True, fancybox=True, fontsize=7, loc='upper left')#, mode='expand')
        ax.set_xlabel(u'$v - v_0$ [km s⁻¹]')
        ax.set_ylabel(u'$I/I_0$ [km s⁻¹]')
        ax = prettify(ax)
    return mle, means, vdf



In [88]:
plt.fill_between?

In [114]:
fig, axes = plt.subplots(1, 1, figsize=(3.5, 3))  # 10 emitters only; 4 absorbers left out.
axes = [axes]#.flatten()
peak_vels_dict = {} 

for k, gal in enumerate([1]):
    print('Now working on {}'.format(wdict[gal].columns.name))
    mle, means, vdf = LyA_vel(wdict, gal, iterations=1000, ax=axes[k])
    peak_velocity = vdf.Centroid.mean()
    peak_vel_stdd = vdf.Centroid.std()
    ax = axes[k]
    vels = mle.data[:, 1]
    real_vel = vels[means.argmax()]
    velsteps = vels.ptp() / (vels.shape[0]-1.)
    # peak_vel_stdd = max(velsteps/2., peak_vel_stdd)  # Is this necessary?
    peak_vels_dict[gal] = [real_vel, peak_velocity, peak_vel_stdd]
PeakVels = pd.DataFrame.from_dict(peak_vels_dict).T
PeakVels.columns = ['PeakVelocity', 'MeanVelocity', 'Stddev']
fig.subplots_adjust(hspace=.02, wspace=.02)
#fig.savefig('LyaPeaks.pdf')
PeakVels

Now working on Haro11C
 
 Estimating uncertainty... 
 Iteration: 1000



,PeakVelocity,MeanVelocity,Stddev
1,158.624373,158.624373,0.79385


In [113]:
vdf.Centroid.hist()

In [115]:
PeakVels.to_csv('LyaPeakVels.csv')

In [116]:
PeakVels = pd.read_csv('LyaPeakVels.csv', index_col=0)
PeakVels

,PeakVelocity,MeanVelocity,Stddev
1,158.624373,158.624373,0.79385


In [118]:
#Velw = pd.DataFrame.from_csv('velwidths.csv')
#Velw = pd.DataFrame.from_csv('velsframe.csv')
#elw.columns = Velw.columns.map(str.strip)
##elw.index = Velw.index.map(lambda x: int(x.split()[-1]))
#rint Velw.columns
#Velw.columns = Velw.columns.map(
#   lambda x: x.replace('/', '').replace('_', ''))
#Velw

Lyman-$\alpha$ peak velocity: 158.6 ± 8 km s⁻¹

In [12]:
lbrand

,col,sym,sty,siz
1,k,s,-,10
2,c,s,-,10
3,m,s,-,10
4,orange,s,-,10
5,k,o,-,11
6,c,o,-,11
7,m,o,-,11
8,orange,o,--,11
9,k,*,--,15
10,c,*,--,15


In [13]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [14]:
CorrFrame = pd.concat([Velw.vint, Velw.vintStd, PeakVels.PeakVelocity], axis=1)#, index=Velw.index)
CorrFrame.corr()

,vint,vintStd,PeakVelocity
vint,1.000000,0.227098,0.441943
vintStd,0.227098,1.000000,0.690228
PeakVelocity,0.441943,0.690228,1.000000


In [15]:
CorrFrame = CorrFrame[CorrFrame.PeakVelocity.notnull()].sort('PeakVelocity')
yy = CorrFrame.vint.dropna()
xx = CorrFrame.PeakVelocity.dropna()
XX = sm.add_constant(xx)
weights = 1/CorrFrame.vintStd.dropna()

re = sm.WLS(yy, XX, weights=weights).fit()
print re.summary()

                            WLS Regression Results                            
Dep. Variable:                   vint   R-squared:                       0.086
Model:                            WLS   Adj. R-squared:                 -0.028
Method:                 Least Squares   F-statistic:                    0.7574
Date:                Wed, 26 Nov 2014   Prob (F-statistic):              0.409
Time:                        01:13:26   Log-Likelihood:                -55.790
No. Observations:                  10   AIC:                             115.6
Df Residuals:                       8   BIC:                             116.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
const         -140.1760     49.181     -2.850   

kurtosistest only valid for n>=20 ... continuing anyway, n=10


In [16]:
st, data, ss2 = summary_table(re, alpha=0.05)
exes = data[:,1]
fittedvalues = data[:,2]
predict_mean_se  = data[:,3]
predict_mean_ci_low, predict_mean_ci_upp = data[:,4:6].T
predict_ci_low, predict_ci_upp = data[:,6:8].T

print max(np.abs(re.fittedvalues - fittedvalues))
#print max(np.abs(iv_l - predict_ci_low))
#print max(np.abs(iv_u - predict_ci_upp))

plot(xx, fittedvalues, 'k--')
plot(xx, yy, 'o', lw=2)
plot(xx, predict_ci_low, 'r--', lw=2)
plot(xx, predict_ci_upp, 'r--', lw=2)
plot(xx, predict_mean_ci_low, 'r--', lw=2)
plot(xx, predict_mean_ci_upp, 'r--', lw=2)
#show()

0.0


In [28]:
import matplotlib.font_manager as fm
#plt.rcdefaults()
#cf_fonts = {'family': 'serif',}
#plt.rc('font', **cf_fonts)
#plt.rc('text', usetex=True)
i, j = 'vint', 'PeakVelocity'
fig = figure(figsize=(3.4, 3.4), dpi=160)
for k in lbrand.index:
    if k in [6, 9, 10, 11]:
        continue
    line = lbrand.ix[k].T
    thing = Velw.ix[k][i]
    gizmo = PeakVels.ix[k][j]
    c = line['col']
    m = line['sym']
    g = line['siz'] * 8
    if i in [9, 10, 11, 12]:
        g *= 1.3
    elif i in [1, 2, 3, 4]:
        g /= 1.2
    plt.errorbar(-thing, gizmo, xerr=Velw.ix[k][i+'Std'], yerr=PeakVels.Stddev.ix[k], ecolor=c, capthick=1., zorder=0)
    scatter(-thing, gizmo, color=c, marker=m, s=g, edgecolor='k', lw=.5, label='{:02d}'.format(k))
    
axhline(y=0, color='k', linestyle='--', lw=.5, zorder=0)
axvline(x=1, color='k', linestyle='--', lw=.5, zorder=0)
tick_params(labelsize=9, axis='both', length=2)
[j.set_linewidth(.6) for j in gca().spines.itervalues()]
#labelsize=9
xlabel(r'$\left|v_{\rm int}^{\rm LIS}\right|$', labelpad=3)#^{Ly\alpha}$')
ylabel(r'$v_{\rm peak}^{\rm Ly \alpha}$')
ax = gca()
#legend()

ax.xaxis.labelpad = 5
ax.yaxis.labelpad = 0
ax.tick_params(labelsize=8)
# Now regression line and confidence bands
#ax.plot(xx, fittedvalues, 'k--', lw=.8, zorder=0)
#ax.plot(predict_mean_ci_low, xx, '--', lw=.5, color='.3', zorder=0)
#ax.plot(predict_mean_ci_upp, xx, '--', lw=.5, color='.3', zorder=0)
zz = linspace(-0, 600, 1000)
ww = zz * re.params['PeakVelocity'] + re.params['const']
ax.plot(-ww, zz, ':', lw=.8, zorder=0, color='.3')
#ax.fill_between(xx, predict_mean_ci_low, predict_mean_ci_upp, color='.3', alpha=.25, zorder=0)
ax.axis((-150, 350, 0, 600))
subplots_adjust(bottom=.12, left=.18,)
leg = legend(loc='upper center', scatterpoints=1, markerscale=.7, ncol=5, fontsize=6, 
             title='LARS ID', fancybox=True, shadow=True).draggable()
prop = fm.FontProperties(size=7)
tit = leg.legend.get_title()
tit.set_fontproperties(prop)
leg.legend.get_frame().set_linewidth(0.6)
fig.subplots_adjust(bottom=.13, top=.95, left=.155, right=.9)
savefig('Paper/Figs/VlyaVint.pdf', facecolor='none', dpi=300)
#savefig('/home/trive/Desktop/VlyaVint.pdf', facecolor='none')
show()
#savefig(savename)
#close()

In [69]:
CorrFrame

,vint,PeakVelocity
1,-100,123.827685
2,-48,149.850253
3,-118,342.855911
4,-31,443.821214
5,-139,165.600560
6,68,NaN
7,-39,178.075840
8,-242,113.914930
9,40,NaN
10,-27,NaN


In [84]:
re.fittedvalues

8     155.822956
14    164.384546
5     223.657090
3     237.487351
13    245.390357
1     249.341860
2     283.588219
7     289.515473
4     294.784144
12    308.614404
dtype: float64

In [89]:
CorrFrame.plot('vint', 'PeakVelocity', style='o')
zz = linspace(-500, 100, 1000)
ww = zz * re.params['vint'] + re.params['const']
plot(zz, ww, 'k-')

In [27]:
re.params

const    315.200242
vint       0.658584
dtype: float64